In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcdefaults()
import warnings
warnings.filterwarnings('ignore')

import datetime as dt
import math, random
from sklearn.metrics import mean_squared_error
#import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import math
from sklearn.metrics import  mean_squared_error


def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

def create_dataset(dataset, time_step=1):
  dataX, dataY= [],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX), np.array(dataY)


def predict():
    df = pd.read_csv('C:/Users/ailnu/Desktop/Product/NSEI.csv')
    col = ['date','open','high','low','close','volume','adjclose']
    count=0
    for i in df.columns:
      df.rename(columns={i:col[count]},inplace=True, errors='raise')
      count=count+1
    df.dropna(axis=0, inplace=True)
    df=remove_outlier(df, 'close')
    df1=df.reset_index()["close"]
    scaler= MinMaxScaler(feature_range=(0,1))
    df1= scaler.fit_transform(np.array(df1).reshape(-1,1))
    training_size= int(len(df1)*0.70)
    test_size= len(df1)-training_size
    train_data, test_data= df1[0:training_size,:], df1[training_size:len(df1),:]
    time_step=7
    Xtrain, ytrain=create_dataset(train_data,time_step)
    xtest,ytest= create_dataset(test_data,time_step)

    Xtrain= Xtrain.reshape(Xtrain.shape[0], Xtrain.shape[1],1)
    xtest= xtest.reshape(xtest.shape[0], xtest.shape[1],1 )
    model=Sequential()

    #Add first LSTM layer
    model.add(LSTM(units=50,return_sequences=True,input_shape=(Xtrain.shape[1],1)))
    #units=no. of neurons in layer
#input_shape=(timesteps,no. of cols/features)
#return_seq=True for sending recc memory. For last layer, retrun_seq=False since end of the line
    model.add(Dropout(0.1))

#Add 2nd LSTM layer
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.1))

#Add 3rd LSTM layer
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.1))

#Add 4th LSTM layer
    model.add(LSTM(units=50))
    model.add(Dropout(0.1))

#Add o/p layer
    model.add(Dense(units=1))

#Compile
    model.compile(optimizer='adam',loss='mean_squared_error')

#Training
    model.fit(Xtrain,ytrain,epochs=50,batch_size=16,verbose=1)

    train_predict=model.predict(Xtrain)
    test_predict= model.predict(xtest)

    train_predict= scaler.inverse_transform(train_predict)
    test_predict= scaler.inverse_transform(test_predict)

    math.sqrt(mean_squared_error(ytrain,train_predict))

    math.sqrt(mean_squared_error(ytest,test_predict))

    dig1= np.empty_like(df1)
    dig1.fill(np.nan)
#print(len(train_predict))
    dig1[time_step+1:len(train_predict)+time_step+1]= train_predict
    dig2= np.empty_like(df1)
    dig2.fill(np.nan)
    print(len(test_predict)," ",len(dig2))
    dig1[time_step+len(train_predict)+time_step+2:]= test_predict

   # plt.figure(figsize=(10,10))
# plt.plot(dig1,'r*')
# plt.plot(dig2,'b')
    df2= scaler.inverse_transform(df1)
#plt.plot(df2,'r')

    input_for_future_prediction= dig1.copy()
    for i in range(len(input_for_future_prediction)):
      if np.isnan(input_for_future_prediction[i]):
       input_for_future_prediction[i]= df2[i]
#plt.plot(input_for_future_prediction,'g',linewidth=2)


    numberOfFuturePrediction = 20
    t=0
    prediction= list()
    while t<numberOfFuturePrediction:
      X= input_for_future_prediction[len(input_for_future_prediction)-time_step:].copy()
  # print(X.shape)
      X= scaler.fit_transform(np.array(X))
      X= X.reshape(1,-1)
      X=X.reshape(1,time_step,1)
      Y= model.predict(np.array(X))

      Y_actual = scaler.inverse_transform(Y)
      prediction.append(Y_actual)
      input_for_future_prediction= np.array(input_for_future_prediction).tolist()
      input_for_future_prediction.append(Y_actual)
      input_for_future_prediction= np.array(input_for_future_prediction)
      t=t+1
      print(prediction)
#plt.plot(input_for_future_prediction,'g',linewidth=2)

      df['date'] = pd.to_datetime(df['date'],format= '%Y-%m-%d' )
      last_date = df['date'].iloc[-1]

    cols = ['Date','Pred','Change','Percent_Change','From','To']
    df3 = pd.DataFrame(columns=cols, index=range(numberOfFuturePrediction))
    count=0
    for i in range(len(df2),len(df2)+numberOfFuturePrediction,1):
        last_date+=dt.timedelta(days=1)
        difference= input_for_future_prediction[i]-input_for_future_prediction[i-1]
        df3.loc[count].Date = last_date
        df3.loc[count].Change = difference
  
        if difference<0:
         df3.loc[count].Pred = 'Down'
        elif difference>0:
            df3.loc[count].Pred = 'Up'
        else:
            df3.loc[count].Pred = 'No change'

            df3.loc[count].Percent_Change = (difference*100/input_for_future_prediction[i-1])
            df3.loc[count].From = input_for_future_prediction[i-1]
            df3.loc[count].To = input_for_future_prediction[i]
            count= count+1
    return df3.to_json(orient ='records')


#print(predict()) 


In [ ]:
  
from flask import Flask, jsonify, request
from flask_cors import CORS
import requests
import json
from bs4 import BeautifulSoup
import re
# helper function
def nse():
    print("alpha")
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 'accept' :'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
    response = requests.get('https://groww.in/stocks/top-gainers', headers = headers, verify = False)
    soup = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile(r"window.__INITIAL_STATE__ = (.*?)$", re.MULTILINE | re.DOTALL)
    script = soup.find("script", text=pattern)
    return json.loads(pattern.search(script.string).group(1))

app = Flask(__name__)   # flask app initialised
CORS(app)               # just to remove CORS error
@app.errorhandler(404)  #error handling for 404
def not_found(e): 
  return jsonify({"status": 0, "error": str(e)})
@app.route("/p")
def scrapeit():
    # only accespts GET requests
    if request.method == "GET": return jsonify({"status": 1, "data": nse()})
    else: return jsonify({"status": 0, "error": "method not callable"})
@app.route("/prediction")
def mod():
    return predict()
if __name__ =="__main__": app.run(debug=True, use_reloader=False) # just remove dubug = True when using it in production


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Epoch 1/50
49/49 [==============================] - 5s 11ms/step - loss: 0.0335
Epoch 2/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0031
Epoch 3/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 4/50
49/49 [==============================] - 1s 10ms/step - loss: 0.0016
Epoch 5/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 6/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 7/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0014
Epoch 8/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0015
Epoch 9/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0016
Epoch 10/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0015
Epoch 11/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 12/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 13/50
49/49 [==================

127.0.0.1 - - [30/Aug/2021 16:29:59] "GET /prediction HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2021 16:29:59] "GET /favicon.ico HTTP/1.1" 200 -


Epoch 1/50
49/49 [==============================] - 5s 12ms/step - loss: 0.0292
Epoch 2/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0024
Epoch 3/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0019
Epoch 4/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0015
Epoch 5/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0015
Epoch 6/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0015
Epoch 7/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0016
Epoch 8/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0015
Epoch 9/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0017
Epoch 10/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0013
Epoch 11/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0013
Epoch 12/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0015
Epoch 13/50
49/49 [==================

...

[array([[13758.9375]], dtype=float32), array([[13754.274]], dtype=float32), array([[13753.935]], dtype=float32), array([[13754.401]], dtype=float32), array([[13756.541]], dtype=float32), array([[13758.962]], dtype=float32), array([[13760.058]], dtype=float32), array([[13756.039]], dtype=float32), array([[13756.073]], dtype=float32), array([[13756.685]], dtype=float32), array([[13757.161]], dtype=float32), array([[13757.519]], dtype=float32), array([[13757.477]], dtype=float32), array([[13757.175]], dtype=float32)]
[array([[13758.9375]], dtype=float32), array([[13754.274]], dtype=float32), array([[13753.935]], dtype=float32), array([[13754.401]], dtype=float32), array([[13756.541]], dtype=float32), array([[13758.962]], dtype=float32), array([[13760.058]], dtype=float32), array([[13756.039]], dtype=float32), array([[13756.073]], dtype=float32), array([[13756.685]], dtype=float32), array([[13757.161]], dtype=float32), array([[13757.519]], dtype=float32), array([[13757.477]], dtype=float32

127.0.0.1 - - [30/Aug/2021 16:30:38] "GET /favicon.ico HTTP/1.1" 200 -


Epoch 1/50
49/49 [==============================] - 5s 12ms/step - loss: 0.0311
Epoch 2/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0024
Epoch 3/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0018
Epoch 4/50
49/49 [==============================] - 1s 12ms/step - loss: 0.0015
Epoch 5/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0014
Epoch 6/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0012
Epoch 7/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0016
Epoch 8/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 9/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 10/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 11/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 12/50
49/49 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 13/50
49/49 [==================

127.0.0.1 - - [30/Aug/2021 16:31:43] "GET /prediction HTTP/1.1" 200 -


[array([[13767.998]], dtype=float32), array([[13761.716]], dtype=float32), array([[13760.448]], dtype=float32), array([[13759.688]], dtype=float32), array([[13761.27]], dtype=float32), array([[13763.708]], dtype=float32), array([[13765.953]], dtype=float32), array([[13762.494]], dtype=float32), array([[13761.744]], dtype=float32), array([[13762.096]], dtype=float32), array([[13762.555]], dtype=float32), array([[13763.057]], dtype=float32), array([[13763.217]], dtype=float32), array([[13762.989]], dtype=float32)]
[array([[13767.998]], dtype=float32), array([[13761.716]], dtype=float32), array([[13760.448]], dtype=float32), array([[13759.688]], dtype=float32), array([[13761.27]], dtype=float32), array([[13763.708]], dtype=float32), array([[13765.953]], dtype=float32), array([[13762.494]], dtype=float32), array([[13761.744]], dtype=float32), array([[13762.096]], dtype=float32), array([[13762.555]], dtype=float32), array([[13763.057]], dtype=float32), array([[13763.217]], dtype=float32), a